# Data Access

In [0]:
service_credential = dbutils.secrets.get(scope="<secret-scope>",key="<service-credential-key>")

spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")

# Database Creation

In [0]:
%sql
CREATE DATABASE gold

# Data Reading and Writing and CRAETING delta tables

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import *

**Storage Variables**

In [0]:
silver = 'abfss://silver@nyctaxistorageansh.dfs.core.windows.net'
gold = 'abfss://gold@nyctaxistorageansh.dfs.core.windows.net'

**DATA ZONE**

In [0]:
df_zone = spark.read.format('parquet')\
                .option('inferSchema',True)\
                .option('header',True)\
                .load(f'{silver}/trip_zone')

In [0]:
df_zone.display()

In [0]:
df_zone.write.format('delta')\
        .mode('append')\
        .option('path',f'{gold}/trip_zone')\
        .saveAsTable('gold.trip_zone')

In [0]:
%sql
select * from gold.trip_zone
where Borough = 'EWR'

**Trip Type**

In [0]:
df_type = spark.read.format('parquet')\
                .option('inferSchema',True)\
                .option('header',True)\
                .load(f'{silver}/trip_type')

In [0]:
df_type.write.format('delta')\
        .mode('append')\
        .option('path',f'{gold}/trip_type')\
        .saveAsTable('gold.trip_type')

**Trips Data**

In [0]:
df_trip = spark.read.format('parquet')\
                .option('inferSchema',True)\
                .option('header',True)\
                .load(f'{silver}/tripsdata')

In [0]:
df_trip.display()

In [0]:
df_trip.write.format('delta')\
        .mode('append')\
        .option('path',f'{gold}/tripsdata')\
        .saveAsTable('gold.trip_trip')

# Learning Delta Lake

**Versioning**

In [0]:
%sql
select * from gold.trip_zone 
where LocationID = 1

In [0]:
%sql
UPDATE gold.trip_zone 
SET Borough = 'EMR' where LocationID = 1;

In [0]:
%sql
DELETE FROM gold.trip_zone 
WHERE LocationID = 1

**Versioning**

In [0]:
%sql
DESCRIBE HISTORY gold.trip_zone

In [0]:
%sql
select * from gold.trip_zone
where LocationID = 1

**Time Travel**

In [0]:
%sql
RESTORE gold.trip_zone TO VERSION AS OF 0

In [0]:
%sql
SELECT * from gold.trip_zone

# Delta Tables

**Trip Type**

In [0]:
%sql
select * from gold.trip_type

**Trip Zone**

In [0]:
%sql
select * from gold.trip_zone

**Trip Data 2023**

In [0]:
%sql
select * from gold.trip_trip